# Bagging and random forests

## Model aggregation

### Question 1

Having $L$ classifiers with a probability of correct prediction $p$ is equivalent to repeating independantly $L$ times a prediction with a probability of success $p$, hence the binomial law.

The probability of correct prediction is:
$\mathbb{P}(X > \frac{L}{2}) = \sum_{i=\frac{L}{2}+1}^{L}\binom{L}{i}p^i(1-p)^{L-i}$

In our case, that means:
$\mathbb{P}(X > 5) = \sum_{i=6}^{10}\binom{10}{i}0.7^i0.3^{10-i}$

In [4]:
from scipy.stats import binom
print((1-binom.cdf(5, 10, 0.7))*100, "%")

84.97316674 %


Even with rather inefficient classifiers ($p=0.7$), making $L=10$ of them vote is enough to get a much better result.

## Bagging

### Question 1